# 02. モデル学習

このノートブックでは、RIMD特徴量を使用してモデルの学習を行います。

## 処理内容
1. 設定の選択・カスタマイズ
2. データローダーの準備
3. モデルの作成
4. 学習の実行
5. 結果の保存

In [ ]:
# 必要なライブラリのインポート
import sys
from pathlib import Path

# プロジェクトルートをパスに追加
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from config.experiment_configs import (
    get_baseline_config, get_cvae_config, get_gnn_config,
    get_large_model_config
)
from src.training.trainer import RIMDTrainer
from src.data.dataset import RIMDDataModule
from src.utils.experiment_manager import ExperimentManager
import logging

# ログレベル設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 実験設定の選択

以下から実験設定を選択してください（コメントアウトを変更）：

In [ ]:
# ===========================================
# 実験設定選択（1つだけコメントアウト解除）
# ===========================================

# 1. ベースライン実験（非VAE MLP）
config = get_baseline_config()

# 2. CVAE実験（不確実性モデリング）
# config = get_cvae_config()

# 3. GNN実験（GraphSAGE）
# config = get_gnn_config()

# 4. 大規模モデル実験
# config = get_large_model_config()

# ===========================================
# カスタム設定例
# ===========================================
# config = get_baseline_config()
# config.exp_id = "custom_experiment"
# config.description = "カスタム実験設定"
# config.model.hidden_dim = 256
# config.loss.lambda_lap = 5e-3
# config.training.learning_rate = 5e-4

print(f"選択された実験: {config.exp_id}")
print(f"説明: {config.description}")
print(f"モデルタイプ: {config.model.model_type}")
print(f"CVAE使用: {config.model.use_cvae}")

In [ ]:
# 実験マネージャーの作成と環境セットアップ
exp_manager = ExperimentManager(config)
exp_manager.setup_experiment()

print(f"実験ディレクトリ: {exp_manager.exp_dir}")
print(f"使用デバイス: {exp_manager.device}")

In [ ]:
# スケーラの読み込み
try:
    scalers = exp_manager.load_scalers()
    print("スケーラ読み込み完了")
    print(f"代表寸法: {scalers['representative_scale']:.2f}")
except FileNotFoundError:
    print("スケーラが見つかりません。01_preprocessing.ipynb を先に実行してください。")
    raise

In [ ]:
# データモジュールの作成
datamodule = RIMDDataModule(config.data, scalers)

# データセット情報表示
print("データセット情報:")
print(f"  Train: {len(datamodule.train_dataset)} cases")
print(f"  Val: {len(datamodule.val_dataset)} cases")
print(f"  Test: {len(datamodule.test_dataset)} cases")

# 特徴量次元確認
edge_dim, node_dim = datamodule.get_feature_dimensions()
print(f"  エッジ特徴量次元: {edge_dim}")
print(f"  ノード特徴量次元: {node_dim}")

In [ ]:
# トレーナーの作成
trainer = RIMDTrainer(config, exp_manager)
print("トレーナー作成完了")

## 学習実行

設定に基づいてモデルの学習を開始します。

In [ ]:
# 学習実行
print("=== 学習開始 ===")
try:
    training_summary = trainer.fit(datamodule)
    print("\n=== 学習完了 ===")
    print(f"学習エポック数: {training_summary['epochs_trained']}")
    print(f"最良検証損失: {training_summary['best_val_loss']:.4f}")
    print(f"最終学習率: {training_summary['final_learning_rate']:.2e}")
    
except KeyboardInterrupt:
    print("\n学習が中断されました")
except Exception as e:
    print(f"\n学習エラー: {e}")
    raise

In [ ]:
# 学習結果の保存
try:
    final_results = {
        **training_summary,
        'experiment_config': config.to_dict()
    }
    
    exp_manager.save_results(final_results)
    print("学習結果保存完了")
    
except Exception as e:
    print(f"結果保存エラー: {e}")

## 簡易評価

学習済みモデルで検証データに対する簡易評価を実行します。

In [ ]:
# 検証データで簡易評価
print("検証データでの簡易評価...")
try:
    val_predictions = trainer.predict(datamodule.val_dataloader())
    
    print(f"予測形状: {val_predictions['predictions'].shape}")
    if 'targets' in val_predictions:
        # 簡易メトリクス計算
        import numpy as np
        pred = val_predictions['predictions']
        target = val_predictions['targets']
        
        mse = np.mean((pred - target) ** 2)
        mae = np.mean(np.abs(pred - target))
        
        print(f"検証MSE: {mse:.4f}")
        print(f"検証MAE: {mae:.4f}")
    
except Exception as e:
    print(f"簡易評価エラー: {e}")

## 学習完了

モデルの学習が完了しました。

### 保存された内容
- ✅ 学習済みモデル（best_model.pth, latest_model.pth）
- ✅ 学習ログ（metrics.jsonl）
- ✅ 実験設定（config.json）
- ✅ 最終結果（final_results.json）

### 次のステップ
1. `03_evaluation.ipynb` で詳細評価
2. `04_analysis.ipynb` で結果分析・可視化

### 別の設定で実験を続ける場合
このセルの設定を変更して再実行してください：
```python
# CVAE実験に変更
config = get_cvae_config()
```